In [ ]:
import shift_detector
from shift_detector.detector import Detector
from shift_detector.checks.dq_metrics_check import DQMetricsCheck
from shift_detector.checks.distinction_check import DistinctionCheck
from shift_detector.checks.sorensen_dice_check import SorensenDiceCheck
from shift_detector.checks.conditional_probabilities_check import ConditionalProbabilitiesCheck
from shift_detector.checks.embedding_distance_check import EmbeddingDistanceCheck
from shift_detector.checks.word_prediction_check import WordPredictionCheck
from shift_detector.checks.statistical_checks.numerical_statistical_check import NumericalStatisticalCheck
from shift_detector.checks.statistical_checks.categorical_statistical_check import CategoricalStatisticalCheck
from shift_detector.checks.statistical_checks.text_metadata_statistical_check import TextMetadataStatisticalCheck
from shift_detector.checks.lda_check import LdaCheck

In [ ]:
import pandas as pd
df = pd.read_csv('./data/pokedex.csv')
df1 = df.head(300)
df2 = df.tail(721-300)

In [ ]:
df1

In [ ]:
df2

In [ ]:
from shift_detector.utils.column_management import ColumnType
custom_column_types = {
    'Legendary': ColumnType.categorical,
    'Type 1': ColumnType.categorical,
    'Type 2': ColumnType.categorical,
    'Generation': ColumnType.categorical
}
detector = Detector(df1, df2, **custom_column_types)

In [ ]:
dqm = DQMetricsCheck()
dc = DistinctionCheck(num_epochs=10, relative_threshold=0.2)
cp = ConditionalProbabilitiesCheck(min_support=0.01, min_confidence=0.01, rule_limit=10,
                                               min_delta_supports=0.01, min_delta_confidences=0.01)
sdc = SorensenDiceCheck(n=1)
edc = EmbeddingDistanceCheck()
wpc = WordPredictionCheck()
nsc = NumericalStatisticalCheck()
csc = CategoricalStatisticalCheck()
tmc = TextMetadataStatisticalCheck()
lc = LdaCheck(n_topics=5, shift_threshold=0.02)

In [ ]:
detector.run(dqm, dc, cp, sdc, edc, wpc, nsc, csc, tmc, lc)

In [ ]:
detector.evaluate()